In [1]:
import matplotlib.pyplot as plt
import scipy as sp
from matplotlib.widgets import Button # , Slider
import threading
import sys
import time
import os
from scipy import interpolate
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib
from GDD import PicoScope as ps  # module for pico_scope
from GDD import ZabberVoiceCoil # module to operate stage 
import numpy as np
from IPython.display import display
%matplotlib notebook


In [6]:
class ProcessGDD(object): #TODO live class
   
    def __init__(self):
        
        #initialize values
        self.n_daq_smp = 1e6
        #self.ps = ps.DAQ_parallel_channels(Samples_per_channel= self.n_daq_smp)
        N = 1e6
        self.timeS = sp.linspace(0,1,N)
        self.signal_HeNe = sp.zeros(N)
        self.signal_SC = sp.zeros(N)
        
    def update_data(self):
        self.timeS,data = self.ps.DAQ_read()
        self.timeS *=1e-9
        self.signal_HeNe = data[:,0]
        self.signal_SC = data[:,1]
     
    def close(self):
        self.ps.DAQ_close()
        
       
        

In [7]:
class LiveOsci(object): #TODO live class
  
    def __init__(self,plot_obj):  #(self#,pico_scope,plot_obj)
        self.process_inst = ProcessGDD()
        self.plot_obj = plot_obj
        global loopBool
        loopBool = False
    
    def getbuttons(self):
        axcolor = 'lightblue'
        ttax = plt.axes([0.05, 0.01, 0.15, 0.06])
        self.buttontt = Button(ttax, 'Single sweep', color=axcolor, hovercolor='0.975')
        self.buttontt.on_clicked(self._update_trace)
        
        runax = plt.axes([0.35, 0.01, 0.05, 0.06])
        self.buttonrun = Button(runax, 'Run', color=axcolor, hovercolor='0.975')
        self.buttonrun.on_clicked(self._looping_pre) #the run command has to be started throgh a thread with a global condition in order to interupt it
        
        stopax = plt.axes([0.425, 0.01, 0.05, 0.06])
        self.buttonstop = Button(stopax, 'Stop', color=axcolor, hovercolor='0.975')
        self.buttonstop.on_clicked(self._stop_meas)

        
    def _looping_pre(self, event): #this is new
        global loopBool
        if (loopBool == False): #this is needed to prevent problems when the runbutton is started twice
            thread=threading.Thread(target=self._run_meas,args=(event,))
            thread.daemon=True   #might or might not be needed
            thread.start()

    def _run_meas(self,event):
        global loopBool
        loopBool = True

        while loopBool:
            data = sp.rand(100)*1000
            self.process_inst.update_data()
            self.plot_obj.update_plotfast1(data)
            time.sleep(0.1)
        
    def _update_trace(self,event):
        data = sp.rand(100)*1000
        self.plot_obj.update_plotfast1(data)
    
    def _stop_meas(self, event):
        global loopBool
        loopBool = False
        self.process_inst.close()

In [8]:
class FastPlotNotebook(object):
    """
    Fast plot using %matplotlib notebook in jupyter notebook
    
    Usage:
    ```
    import time
    updatepl =  FastPlotNotebook(wl1,Td)
    for layer,T,Tbefore in zip(n[1:],Td[1:],Td[:-1]):
        updatepl.update_plot(wl1,Tbefore,T)
        time.sleep(0.2)
    ```
    """
    def __init__(self):
         
        self.p = p
        fig,ax1 = plt.subplots(1,figsize=(15/2.54, 15/2.54))
        self.fig = fig
        self.ax1 = ax1
        self.ax1.set_title('HeNe, IR')
        self.ax1.set_xlim(0,1)
        self.ax1.set_ylim(-10,10)
        self.ax1.set_xlabel('Time (s)')
        self.ax1.set_ylabel('Signal (a.u.)')     
        timeS = self.p.timeS
        signal_HeNe = self.p.signal_HeNe
        signal_SC = self.p.signal_SC
         
        self.HeNe_pl, = self.ax1.plot(timeS[::100],signal_HeNe[::100])
        self.SC_pl, = self.ax1.plot(timeS[::100],signal_SC[::100])
        
        
        self.fig.tight_layout()
        self.fig.subplots_adjust(wspace = 0.7,bottom = 0.2)
    def update_plot(self,x1,y1,x2,y2):
        self.line1.set_data(x1,y1)
        self.line2.set_data(x2,y2)
        self.ax1.set_xlim(x1.min(),x1.max())
        self.ax1.set_ylim(-1000,1000)
        self.ax2.set_xlim(x2.min(),x2.max())
        self.ax2.set_ylim(-1000,1000)
        self.fig.canvas.draw()
        self.fig.tight_layout()
        self.fig.subplots_adjust(wspace = 0.7,bottom = 0.2)
 

In [9]:

plot_obj = FastPlotNotebook()
# rp =  ps.DAQ_parallel_channels()

t = sp.linspace(0,1,100)


plot_obj.update_plot(t,t,t,t*2)
liveplt = LiveOsci(plot_obj) #rp,rpplot
liveplt.getbuttons()

NameError: name 'p' is not defined

In [56]:
plot_obj.fig.subplots_adjust(wspace = 0.7,bottom = 0.2)

In [10]:
rp.DAQ_close()

In [1]:
2**16

65536